# **Evaluation**

**Intrinsic-Word similarity**

1. Import dependancies

In [ ]:
import numpy as np
import scipy
from scipy.stats import *
import sys

2. define data-load and calculation functions 

In [ ]:
embeddings = ""


def loadData():
    ret = {}
    # I use Colab here so I simply copy the data's path, I'll upload all the data this notebook need, if the path doesn't work, please change it manualy.
    #I comment the other spine embeddings, while evaluate other word embeddings please remove the "#" sign and comment present evaluated word embeddings.
    data = open("../input/spine-embeddings/glove.6B.300d.txt.spine","r").readlines()#glove
    #data = open("../input/spine-embeddings/GoogleNews-vectors-negative300(first500000).txt.spine","r").readlines()#Word2Vec
    #data = open("../input/spine-embeddings/wiki-news-300d-1M.txt.spine","r").readlines()#Fasttext
    #data = open("../input/spine-embeddings/sentence_embeddings.txt.spine","r").readlines()#sentence embedding
    #data = open("../input/spine-embeddings/image_embeddings.txt.spine","r").readlines()#image embedding
    
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.15_bs_64_mm_0.spine","r").readlines()#learning rate = 0.15, batch size = 64, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.01_bs_64_mm_0.spine","r").readlines()#learning rate = 0.01, batch size = 64, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.001_bs_64_mm_0.spine","r").readlines()#learning rate = 0.001, batch size = 64, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.0001_bs_64_mm_0.spine","r").readlines()#learning rate = 0.0001, batch size = 64, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_16_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 16, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_32_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 32, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_128_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 128, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_256_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 256, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.1.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.1
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.3.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.3
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.6.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.6
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.9.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.9
    
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_without_psl.spine","r").readlines()#without psl loss
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_without_asl.spine","r").readlines()#without asl loss
    for row in data:
        word = row.strip().split(' ')[0]
        vals = row.strip().split(' ')[1:]
        vals = np.array( [float(val) for val in vals] )
        ret[word] = vals
    return ret


def loadTestData():
    # I use Colab here so I simply copy the data's path, I'll upload all the data this notebook need, if the path doesn't work, please change it manualy.
    tmp = open("../input/word-sim/word_sim.tab","r").readlines()
    data = {}
    data['words'] = [ row.strip().split('\t')[0:2] for i,row in enumerate(tmp) if i!=0 ]
    data['sim_scores'] = [ float(row.strip().split('\t')[2]) for i,row in enumerate(tmp) if i!=0  ]
    return data

def getSimilarity(e1, e2):
    # cosine similarity
    return np.sum(e1 * e2)/( np.sqrt(np.sum(e1*e1)) * np.sqrt(np.sum(e2*e2)))

def getSimilarityScoreForWords(w1,w2):
	global embeddings
	#print w1
	#print embeddings
	if (w2 not in embeddings) or (w1 not in embeddings) :
		return -1
	finalVector_w1 = embeddings[w1]
	finalVector_w2 = embeddings[w2]
	return getSimilarity(finalVector_w1, finalVector_w2)

def evaluate():
	global embeddings
	print("--- loading data...")
	embeddings = loadData()
	data = loadTestData()
	print("#words = ", len(data['words']))
	print("#scores = ", len(data['sim_scores']))
	print("--- checking...")
	pred_scores = []
	invalid = 0
	pred_scores = [ [getSimilarityScoreForWords(w1w2[0],w1w2[1]),human_score] for w1w2,human_score in zip(data['words'], data['sim_scores']) ]			
	pred_scores = np.array( [ val for val in pred_scores if val[0]!=-1 ] )
	#print pred_scores
	spearman_rank_coeff,sp_rho = spearmanr( pred_scores[:,0], pred_scores[:,1] )
	print("total, valid,spearman_rank_coeff,sp_rho ", len(data['words']),len(pred_scores), spearman_rank_coeff,sp_rho)


In [ ]:
evaluate()

**extrinsic_downstream**

*newsgroups*

1. get evaluate data

In [ ]:
from nltk import word_tokenize
from sklearn.datasets import fetch_20newsgroups
import pickle
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import nltk
nltk.download('punkt')

computer_categories = ['comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware']
religion_categories = ['alt.atheism', 'soc.religion.christian']

def get_Xy(data):
    X, y = [], []
    for idx in range(len(data['data'])):
        X.append(word_tokenize(data['data'][idx].lower()))
        y.append(data['target'][idx])
    return X, y

# I use Colab here so I simply copy the data's path, I'll upload all the data this notebook need, if the path doesn't work, please change it manualy.
def get_everything(categories, val_fraction, name, data_dir="./"):
    data_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
    data_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
    data_test_X, data_test_y = get_Xy(data_test)
    data_X, data_y = get_Xy(data_train)
    
    print("len(data_X):",len(data_X))
    print("len(data_test_X):",len(data_test_X))
    val_threshold = int( (1.0 - val_fraction)* len(data_X) )

    data_train_X, data_train_y = data_X[:val_threshold], data_y[:val_threshold]
    data_val_X, data_val_y = data_X[val_threshold:], data_y[val_threshold:]
    
    pickle.dump(data_train_X, open(data_dir+name+"_train_X.p", 'wb'))
    pickle.dump(data_train_y, open(data_dir+name+"_train_y.p", 'wb'))
    pickle.dump(data_val_X, open(data_dir+name+"_val_X.p", 'wb'))
    pickle.dump(data_val_y, open(data_dir+name+"_val_y.p", 'wb'))
    pickle.dump(data_test_X, open(data_dir+name+"_test_X.p", 'wb'))
    pickle.dump(data_test_y, open(data_dir+name+"_test_y.p", 'wb'))

    
get_everything(computer_categories,0.2,'news_computer')
get_everything(religion_categories,0.2,'news_religion')




2. evaluate

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
#from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels import RBF
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
import sys
import pickle

h = .02  # step size in the mesh
embedding_size = None
vectors = None

# python classify.py embedding_src num_classes x_train_pickle y_train_pickle x_val_pickle y_val_pickle x_test_pickle y_test_pickle

best_val = 0.0
best_test = 0.0

def loadVectors():
    global embedding_size
    global vectors
    # I use Colab here so I simply copy the data's path, I'll upload all the data this notebook need, if the path doesn't work, please change it manualy.
    #I comment the other spine embeddings, while evaluate other word embeddings please remove the "#" sign and comment present evaluated word embeddings.
    data = open("../input/spine-embeddings/glove.6B.300d.txt.spine","r").readlines()#glove
    #data = open("../input/spine-embeddings/GoogleNews-vectors-negative300(first500000).txt.spine","r").readlines()#Word2Vec
    #data = open("../input/spine-embeddings/wiki-news-300d-1M.txt.spine","r").readlines()#Fasttext
    #data = open("../input/spine-embeddings/sentence_embeddings.txt.spine","r").readlines()#sentence embedding
    #data = open("../input/spine-embeddings/image_embeddings.txt.spine","r").readlines()#image embedding
    
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.15_bs_64_mm_0.spine","r").readlines()#learning rate = 0.15, batch size = 64, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.01_bs_64_mm_0.spine","r").readlines()#learning rate = 0.01, batch size = 64, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.001_bs_64_mm_0.spine","r").readlines()#learning rate = 0.001, batch size = 64, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.0001_bs_64_mm_0.spine","r").readlines()#learning rate = 0.0001, batch size = 64, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_16_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 16, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_32_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 32, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_128_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 128, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_256_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 256, momentum = 0
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.1.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.1
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.3.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.3
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.6.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.6
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.9.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.9
    
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_without_psl.spine","r").readlines()#without psl loss
    #data = open("../input/spine-embeddings/glove.6B.300d.txt_without_asl.spine","r").readlines()#without asl loss
    vectors = {}
    for row in data:
        vals = row.split()
        word = vals[0]
        vals = np.array( [float(val) for val in vals[1:]] )
        vectors[word] = vals
    embedding_size = len(vals)
    print("embedding_size = ",embedding_size)
    return vectors


def getFeats(sentence):
    global vectors
    global embedding_size
    ret = np.zeros(embedding_size)
    cnt = 0
    for word in sentence:
        if word.lower() in vectors:
            ret+=vectors[word.lower()]
            cnt+=1
    if cnt>0:
        ret/=cnt
    return ret

def getOneHot(vals, max_num):
    ret = np.zeros((vals.shape[0], max_num))
    for i,val in enumerate(vals):
        ret[i][val] = 1
    return ret

def trainAndTest(x_splits, y_splits, clf):
    global best_val, best_test
    # print("x_splits[0] = ", x_splits[0])
    # print("x_splits[0] = ", type(x_splits[0]))
    clf.fit(x_splits[0], y_splits[0])
    flag = False
    if len(x_splits[1])>0:
        score = clf.score(x_splits[1], y_splits[1])
        if score > best_val:
            flag = True
            best_val = score
        print("Val Score = ", score)
    score = clf.score(x_splits[2], y_splits[2])
    if flag:
        best_test = score
    print("Test Score = ", score)

def main():
    loadVectors()
    num_classes = 2
    print("num_classes = ",num_classes)
    classifiers = None
    if num_classes==2:
        classifiers = [
        SVC(kernel="linear", C=0.025, class_weight='balanced'),
        SVC(kernel="linear", C=0.1, class_weight='balanced'),
        SVC(kernel="linear", C=5, class_weight='balanced'),
        SVC(kernel="linear", C=10, class_weight='balanced'),
        SVC(kernel="linear", C=50, class_weight='balanced'),
        SVC(kernel="linear", C=100, class_weight='balanced'),
        SVC(kernel="linear", C=500, class_weight='balanced'),
        SVC(kernel="linear", C=1000, class_weight='balanced'),
        SVC(kernel="linear", C=0.25, class_weight='balanced'),
        SVC(gamma=2, C=0.1, class_weight='balanced'),
        SVC(gamma=2, C=0.25, class_weight='balanced'),
        SVC(C=0.1, class_weight='balanced'),
        SVC(C=5, class_weight='balanced'),
        SVC(C=10, class_weight='balanced'),
        SVC(C=50, class_weight='balanced'),
        SVC(C=100, class_weight='balanced'),
        SVC(C=500, class_weight='balanced'),
        SVC(C=1000, class_weight='balanced'),
        SVC(class_weight='balanced'),
        MLPClassifier(alpha=1),
        GaussianNB(),
        RandomForestClassifier()]
    else:
        #TODO
        pass


    all_feats = []
    labels = []
    idx = 3
    # I use Colab here so I simply copy the data's path, I'll upload all the data this notebook need, if the path doesn't work, please change it manualy.
    while idx<8:
        if idx == 3:
            texts = pickle.load( open("./news_computer_train_X.p","rb") )
        if idx == 5:
            texts = pickle.load( open("./news_computer_test_X.p","rb") )
        if idx == 7:
            texts = pickle.load( open("./news_computer_val_X.p","rb") )
        if len(texts)>0:
            feats = np.array( [getFeats(t) for t in texts] )
            print("feats : ",feats.shape)
            all_feats.append( feats )
            idx+=1
            if idx == 4:
                cur_labels = np.array(pickle.load( open("./news_computer_train_y.p","rb") ) )
            if idx == 6:
                cur_labels = np.array(pickle.load( open("./news_computer_test_y.p","rb") ) )
            if idx == 8:
                cur_labels = np.array(pickle.load( open("./news_computer_val_y.p","rb") ) )
                
            #cur_labels = np.array(pickle.load( open(sys.argv[idx],"rb") ) )
            #cur_labels = getOneHot(cur_labels, max(cur_labels)+1)
            labels.append( cur_labels )
            print("cur_labels : ",cur_labels.shape)
            idx+=1
        else:
            idx+=2
            labels.append([])
            all_feats.append([])

    for clf in classifiers:
        #print "="*33
        #print "clf = ",clf
        trainAndTest(all_feats, labels, clf)

print("on going")
main()
print('best val', best_val)
print('best test', best_test)

*np-bracketing*

1. get evaluate data 

In [ ]:
!wget http://angelikilazaridou.github.io/resourses/parsing/NP_dataset.tar.gz
!tar -xvzf NP_dataset.tar.gz

In [ ]:
import pickle
import numpy
data = open("dataset.txt","r").readlines()[1:]
data_dir = "./"
name = "npbracketing"

x_splits = {i:[] for i in range(10)}
y_splits = {i:[] for i in range(10)}
target_dict = {'R':0,'L':1}
for row in data:
    vals = row.strip().split()
    split = int(vals[0])
    target = target_dict[vals[-1]]
    x_splits[split].append( [word_pos.split('-')[0] for word_pos in vals[1].split('_') ] )
    y_splits[split].append(target)

for i in range(10):
    print(len(x_splits[i]), x_splits[i][0])
    print(len(y_splits[i]), y_splits[i][0])
    x_train = x_splits[i]
    y_train = y_splits[i]
    x_val = []
    y_val = []
    x_test = []
    y_test = []
    for j in range(10):
        if j!=i:
            x_test.extend(x_splits[j])
            y_test.extend(y_splits[j])
    pickle.dump(x_train,open(data_dir+name+"_"+"train_X"+str(i)+".pickle","wb"))
    pickle.dump(x_val,open(data_dir+name+"_"+"val_X" + str(i) + ".pickle","wb"))
    pickle.dump(x_test,open(data_dir+name+"_"+"test_X" + str(i) + ".pickle","wb"))
    pickle.dump(y_train,open(data_dir+name+"_"+"train_y"+ str(i) + ".pickle","wb"))
    pickle.dump(y_val,open(data_dir+name+"_"+"val_y"+ str(i) + ".pickle","wb"))
    pickle.dump(y_test,open(data_dir+name+"_"+"test_y" + str(i) + ".pickle","wb"))

In [ ]:
for i in range(2):
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.colors import ListedColormap
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.datasets import make_moons, make_circles, make_classification
    from sklearn.neural_network import MLPClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC
    from sklearn.gaussian_process import GaussianProcessClassifier
    from sklearn.gaussian_process.kernels import RBF
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
    from sklearn.ensemble import RandomForestClassifier
    import sys
    import pickle

    h = .02  # step size in the mesh
    embedding_size = None
    vectors = None

    # python classify.py embedding_src num_classes x_train_pickle y_train_pickle x_val_pickle y_val_pickle x_test_pickle y_test_pickle

    def loadVectors():
        global embedding_size
        global vectors
        # I use Colab here so I simply copy the data's path, I'll upload all the data this notebook need, if the path doesn't work, please change it manualy.
        #I comment the other spine embeddings, while evaluate other word embeddings please remove the "#" sign and comment present evaluated word embeddings.
        data = open("../input/spine-embeddings/glove.6B.300d.txt.spine","r").readlines()#glove
        #data = open("../input/spine-embeddings/GoogleNews-vectors-negative300(first500000).txt.spine","r").readlines()#Word2Vec
        #data = open("../input/spine-embeddings/wiki-news-300d-1M.txt.spine","r").readlines()#Fasttext
        #data = open("../input/spine-embeddings/sentence_embeddings.txt.spine","r").readlines()#sentence embedding
        #data = open("../input/spine-embeddings/image_embeddings.txt.spine","r").readlines()#image embedding

        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.15_bs_64_mm_0.spine","r").readlines()#learning rate = 0.15, batch size = 64, momentum = 0
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.01_bs_64_mm_0.spine","r").readlines()#learning rate = 0.01, batch size = 64, momentum = 0
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.001_bs_64_mm_0.spine","r").readlines()#learning rate = 0.001, batch size = 64, momentum = 0
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.0001_bs_64_mm_0.spine","r").readlines()#learning rate = 0.0001, batch size = 64, momentum = 0
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_16_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 16, momentum = 0
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_32_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 32, momentum = 0
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_128_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 128, momentum = 0
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_256_mm_0.spine","r").readlines()#learning rate = 0.1, batch size = 256, momentum = 0
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.1.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.1
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.3.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.3
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.6.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.6
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_lr0.1_bs_64_mm_0.9.spine","r").readlines()#learning rate = 0.1, batch size = 64, momentum = 0.9

        #data = open("../input/spine-embeddings/glove.6B.300d.txt_without_psl.spine","r").readlines()#without psl loss
        #data = open("../input/spine-embeddings/glove.6B.300d.txt_without_asl.spine","r").readlines()#without asl loss
        vectors = {}
        for row in data:
            vals = row.split()
            word = vals[0]
            vals = np.array( [float(val) for val in vals[1:]] )
            vectors[word] = vals
        embedding_size = len(vals)
        #print "embedding_size = ",embedding_size
        return vectors


    def getFeats(sentence):
        global vectors
        global embedding_size
        if False:
            ret = np.zeros(embedding_size)
            cnt = 0
            for word in sentence:
                if word in vectors:
                    ret+=vectors[word]
                    cnt+=1
            if cnt>0:
                ret/=cnt
            return ret
        else:
            ret = []
            cnt = 0
            for word in sentence:
                if word in vectors:
                    ret.extend([ v for v in vectors[word]])
                    cnt+=1
                else:
                    ret.extend( [v for v in np.zeros(embedding_size)] )
            ret = np.array(ret)
            return ret

    def getOneHot(vals, max_num):
        ret = np.zeros((vals.shape[0], max_num))
        for i,val in enumerate(vals):
            ret[i][val] = 1
        return ret

    def trainAndTest(x_splits, y_splits, clf):
        clf.fit(x_splits[0], y_splits[0])
        train_score = clf.score(x_splits[0], y_splits[0])
        val_score = None
        if len(x_splits[1])>0:
            val_score = clf.score(x_splits[1], y_splits[1])
            #print "Val Score = ", val_score
        score = clf.score(x_splits[2], y_splits[2])
        #print "Test Score = ", score
        return train_score,val_score,score

    def main():
        loadVectors()
        num_classes = 2
        #print "num_classes = ",num_classes
        classifiers = None
        if num_classes==2:
            classifiers = [
            SVC(kernel="linear", C=0.025),
            SVC(kernel="linear", C=0.1),
            SVC(kernel="linear", C=1.0),
            SVC(gamma=2, C=1),
            RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
            RandomForestClassifier(max_depth=5, n_estimators=50, max_features=10),
            MLPClassifier(alpha=1),
            RandomForestClassifier(n_estimators=20, max_features=10)]
        else:
            #TODO
            pass


        all_feats = []
        labels = []
        idx = 3
        while idx<8:
            if i == 0 and idx == 3:
                texts = pickle.load( open("./npbracketing_train_X0.pickle","rb") )
            if i == 0 and idx == 5:
                texts = pickle.load( open("./npbracketing_val_X0.pickle","rb") )
            if i == 0 and idx == 7:
                texts = pickle.load( open("./npbracketing_test_X0.pickle","rb") )
            if i == 1 and idx == 3:
                texts = pickle.load( open("./npbracketing_train_X1.pickle","rb") )
            if i == 1 and idx == 5:
                texts = pickle.load( open("./npbracketing_val_X1.pickle","rb") )
            if i == 1 and idx == 7:
                texts = pickle.load( open("./npbracketing_test_X1.pickle","rb") )
            #texts = pickle.load( open(sys.argv[idx],"rb") )
            if len(texts)>0:
                feats = np.array( [getFeats(t) for t in texts] )
                #print "feats : ",feats.shape
                all_feats.append( feats )
                idx+=1
                if i == 0 and idx == 4:
                    cur_labels = pickle.load( open("./npbracketing_train_y0.pickle","rb") )
                if i == 0 and idx == 6:
                    cur_labels = pickle.load( open("./npbracketing_val_y0.pickle","rb") )
                if i == 0 and idx == 8:
                    cur_labels = pickle.load( open("./npbracketing_test_y0.pickle","rb") )
                if i == 1 and idx == 4:
                    cur_labels = pickle.load( open("./npbracketing_train_y1.pickle","rb") )
                if i == 1 and idx == 6:
                    cur_labels = pickle.load( open("./npbracketing_val_y1.pickle","rb") )
                if i == 1 and idx == 8:
                    cur_labels = pickle.load( open("./npbracketing_test_y1.pickle","rb") )
                #cur_labels = np.array(pickle.load( open(sys.argv[idx],"rb") ) )
                #cur_labels = getOneHot(cur_labels, max(cur_labels)+1)
                labels.append( cur_labels )
                #print "cur_labels : ",cur_labels.shape
                idx+=1
            else:
                idx+=2
                labels.append([])
                all_feats.append([])
        print("Done loading data")

        best_test = 0.0
        best_clf = None
        best = 0.0
        for clf in classifiers:
            #print "="*33
            #print "clf = ",clf
            score, val_score, test_score = trainAndTest(all_feats, labels, clf)
            best_test = max(best_test, test_score)
            if score>best:
                best = score
                best_clf = clf
        print("best_test for this split= ", best_test)
        #print "best_test = ", best_test
        #print "best= ", best, best_clf

    main()
